## Dynamic Generation of Transducible Functions

In [ ]:
from agentics.core.transducible_functions import make_transducible_from_types, Transduce, With
from pydantic import BaseModel
from typing import Optional
from agentics.core.atype import *

class GenericInput(BaseModel):
    content: Optional[str] = None

class Email(BaseModel):
    to: Optional[str] = None
    subject:Optional[str]=None
    body: Optional[str]=None

### Dynamic Generation

Transducible functions can be dynamically generared in two ways:
1) Using make_transducible_from_types method
2) Using << operator between any two pydantic types

In [ ]:

fun = make_transducible_from_types(GenericInput,Email, instructions="Write an email about the provided content and elaborate on that")
print(await fun(GenericInput(content="Hi Lisa, I just released agentics 2.0.")))

## Composition


In [ ]:
class Summary(BaseModel):
    summary_text:Optional[str]
 
input = GenericInput(content="Write news story on Zoran Mandani won the Election in NYC and send it to Alfio")
write_mail = Email<<GenericInput
mail = await write_mail(input)
print(mail.model_dump_json(indent=2))

summary_composite_1 = Summary << write_mail
mail = await summary_composite_1(input)
print(mail.model_dump_json(indent=2))

summary_composite_2 = Summary <<(Email<<GenericInput)
mail = await summary_composite_2(input)
print(mail.model_dump_json(indent=2))

In [ ]:
summarize = Summary << With(
    Email,
    instructions="Rewrite the email into a summary",
    #llm="watsonx/openai/gpt-oss-120b",
    enforce_output_type=True,
    verbose_transduction=False
)
write_mail =(Email<<GenericInput)
mail = await write_mail(GenericInput(content="Zoran Mandani won the Election in NYC"))
print(await summarize(mail))